In [21]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
database= "sql-murder-mystery.db"

In [23]:
connection = sqlite3.connect(database)
cur = connection.cursor()

In [24]:
#Making a function to facilitate running queries
def runquery(query):
    res = cur.execute(query)
    return res.fetchall()

def runquerylist(queries):
    results = []
    for query in queries:
        results.append(runquery(query))
    for i, result in enumerate(results):
        for row in result:
            print(row)
        print("\n")
    

In [25]:
#The following query uses information from the prompt to pull up the crime scene report
runquery("select * from crime_scene_report where city='SQL City' and date = 20180115 and type = 'murder' ")

[(20180115,
  'murder',
  'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".',
  'SQL City')]

In [26]:
#Trying to find the first witness from the report. Assuming that the last house has the highest address number
runquery("select * from person where address_street_name = 'Northwestern Dr' order by address_number desc limit 1")

[(14887, 'Morty Schapiro', 118009, 4919, 'Northwestern Dr', 111564949)]

In [28]:
#From the above quere, we get to know that the person_id of the first wittness is 14887
runquery("select transcript from interview where person_id = 14887")

[('I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".',)]

In [29]:
#Now, we try to find the interview of the second witness in the same way to gather more clues about the murderer
runquery("select * from person where address_street_name = 'Franklin Ave' and name = 'Annabel' ")


[]

In [30]:
#Since the above does not return anything, maybe Annabel is not the full name of the person
runquery("select * from person where address_street_name = 'Franklin Ave' and person.name like 'Annabel%' ")

[(16371, 'Annabel Miller', 490173, 103, 'Franklin Ave', 318771143)]

In [31]:
runquery("select transcript from interview where person_id = 16371")

[('I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.',)]

In [32]:
#Checking if facebook activity of the winesses has any useful clues.
runquerylist(["select * from facebook_event_checkin where person_id=14887",
              "select * from facebook_event_checkin where person_id=16371"])

(14887, 4719, 'The Funky Grooves Tour', 20180115)


(16371, 4719, 'The Funky Grooves Tour', 20180115)




In [33]:
runquery("select * from get_fit_now_check_in where membership_id = '48Z7A' ")

# and id = (select membership_id from get_fit_now_check_in where check_in_date = 20180109)

[('48Z7A', 20180109, 1600, 1730)]

In [34]:
#No useful clue was found while checking acebook activity. Now, we use all the other useful clues to try to find the murderer
#Uptil now, we know that the membership number of the person started with 48Z, that he had a gold memebrship and he was working out on January 9 2018
runquery("select * from get_fit_now_member where membership_status = 'gold' and id like '48Z%' and id in (select membership_id from get_fit_now_check_in where check_in_date = 20180109)")

[('48Z55', 67318, 'Jeremy Bowers', 20160101, 'gold'),
 ('48Z7A', 28819, 'Joe Germuska', 20160305, 'gold')]

In [35]:
#Using the previous query, we narrowed the suspects down to 2. Now we also know that the number plate of the murderer's car included H42W so we use that.
runquery("select * from drivers_license where plate_number like '%H42W%' and id = (select license_id from person where id = (select person_id from get_fit_now_member where id = '48Z55'))")
#Notice that we designed the query such that we only get a result if the suspect fits all the clues. If it does not fit the license plate clue, we get nothing.

[(423327, 30, 70, 'brown', 'brown', 'male', '0H42W2', 'Chevrolet', 'Spark LS')]

In [36]:
runquery("select * from drivers_license where plate_number like '%H42W%' and id = (select license_id from person where id = (select person_id from get_fit_now_member where id = '48Z7A'))")
#Since we got a result for the first query but not second, we conclude that the persno with gym membership id '48Z55' is the murderer.

[]

In [37]:
#This was designed by the challenge designer to check your answer
runquery("INSERT INTO solution VALUES (1, 'Jeremy Bowers')")
        
runquery("SELECT value FROM solution")

[("Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.",)]

In [38]:
#From our previous query where we found the gym membership information of Jeremy Bowers, we can see that his person_id is 67318. We use this to find his interview.
runquery("select * from interview where person_id = 67318")

[(67318,
  'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n')]

In [39]:
#A single query that looks through the database for the persson that satisfies all the clues given by the murderer.
runquery("select * from person where id = (select person_id from facebook_event_checkin where event_name = 'SQL Symphony Concert' and cast(date as varchar) like '201712%' intersect select id from person where license_id = (select id from drivers_license where 65<=height and height<=67 and hair_color = 'red' and gender = 'female' and car_make = 'Tesla' and car_model = 'Model S')) ")

[(99716, 'Miranda Priestly', 202298, 1883, 'Golden Ave', 987756388)]

In [40]:
#Checking our solution again.
runquery("INSERT INTO solution VALUES (1, 'Miranda Priestly')")
        
runquery("SELECT value FROM solution")

[('Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!',)]

In [41]:
#This is used to revert all the changes made to the database using the insert command so it can reused for the challenge.
connection.rollback()